In [ ]:
# !pip install pandas scikit-learn flask nest_asyncio

In [1]:
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import numpy as np

class RecommendationSystem:
    def __init__(self):
        self.movies_df = self.load_data()
        self.cosine_sim, self.cosine_sim2 = self.build_similarity_matrices()

    def load_data(self):
        movies_df = pd.read_csv("tmdb_5000_movies.csv")
        credits_df = pd.read_csv("tmdb_5000_credits.csv")
        credits_df.columns = ['id', 'title', 'cast', 'crew']
        merged_df = movies_df.merge(credits_df, on='id')
        merged_df['title'] = merged_df['title_x'].combine_first(merged_df['title_y'])
        merged_df.drop(columns=['title_x', 'title_y'], inplace=True)
        return merged_df

    def weighted_rating(self, x, C, m):
        v = x["vote_count"]
        R = x["vote_average"]
        return (v / (v + m) * R) + (m / (v + m) * C)

    def build_similarity_matrices(self):
        C = self.movies_df["vote_average"].mean()
        m = self.movies_df["vote_count"].quantile(0.9)
        new_movies_df = self.movies_df.copy().loc[self.movies_df["vote_count"] >= m]
        new_movies_df["score"] = new_movies_df.apply(lambda x: self.weighted_rating(x, C, m), axis=1)
        new_movies_df = new_movies_df.sort_values('score', ascending=False)

        tfidf = TfidfVectorizer(stop_words="english")
        self.movies_df["overview"] = self.movies_df["overview"].fillna("")
        tfidf_matrix = tfidf.fit_transform(self.movies_df["overview"])
        cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

        for feature in ['cast', 'crew', 'keywords', 'genres']:
            self.movies_df[feature] = self.movies_df[feature].apply(literal_eval)

        def get_director(x):
            for i in x:
                if i["job"] == "Director":
                    return i["name"]
            return np.nan

        def get_list(x):
            if isinstance(x, list):
                names = [i["name"] for i in x]
                if len(names) > 3:
                    names = names[:3]
                return names
            return []

        self.movies_df["director"] = self.movies_df["crew"].apply(get_director)

        for feature in ['cast', 'keywords', 'genres']:
            self.movies_df[feature] = self.movies_df[feature].apply(get_list)

        self.movies_df = self.movies_df.reset_index()

        def clean_data(x):
            if isinstance(x, list):
                return [str.lower(i.replace(" ", "")) for i in x]
            else:
                if isinstance(x, str):
                    return str.lower(x.replace(" ", ""))
                else:
                    return ""

        features = ['cast', 'keywords', 'director', 'genres']
        for feature in features:
            self.movies_df[feature] = self.movies_df[feature].apply(clean_data)

        def create_soup(x):
            return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

        self.movies_df["soup"] = self.movies_df.apply(create_soup, axis=1)
        count_vectorizer = CountVectorizer(stop_words="english")
        count_matrix = count_vectorizer.fit_transform(self.movies_df["soup"])
        cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

        return cosine_sim, cosine_sim2

    def get_recommendations(self, title, cosine_sim):
        indices = pd.Series(self.movies_df.index, index=self.movies_df["title"]).drop_duplicates()
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movies_indices = [i[0] for i in sim_scores]
        return self.movies_df["title"].iloc[movies_indices]


C:\Users\pantv\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\pantv\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
movies_df = pd.read_csv("tmdb_5000_movies.csv")
movies_df.title

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: title, Length: 4803, dtype: object

In [ ]:
from flask import Flask, request, render_template_string
import nest_asyncio

nest_asyncio.apply()

app = Flask(__name__)
rec_sys = RecommendationSystem()

# Define the HTML template with a background video and advanced animations
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Movie Recommendation System</title>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');
        body {
            font-family: 'Roboto', sans-serif;
            margin: 0;
            padding: 0;
            background:	#F08080;
            color: #ecf0f1;
            overflow-x: hidden;
        }
        #background-video {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            object-fit: cover;
            z-index: -1;
            filter: brightness(50%);
        }
        .content {
            text-align: left;
            z-index: 1;
            animation: fadeIn 2s ease-out;
            padding: 20px;
            width: 100%;
            max-width: 800px;
            margin: 0 auto;
        }
        h1 {
            font-size: 3.5em;
            color: #87CEFA;
            animation: fadeInDown 2s ease-out;
            text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.4);
            margin-bottom: 20px;
        }
        form {
            background: rgba(0, 0, 0, 0.8);
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.7);
            animation: fadeInUp 2s ease-out;
            max-width: 500px;
            width: 100%;
            margin: 20px auto;
        }
        label {
            font-size: 1.2em;
            margin-bottom: 10px;
            display: block;
        }
        input[type="text"] {
            width: 100%;
            padding: 15px;
            border: none;
            border-radius: 5px;
            margin-top: 10px;
            margin-bottom: 20px;
            font-size: 1em;
        }
        input[type="submit"] {
            background-color: #e74c3c;
            color: white;
            padding: 15px 30px;
            border: none;
            border-radius: 5px;
            font-size: 1.2em;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }
        input[type="submit"]:hover {
            background-color: #c0392b;
        }
        ul {
            list-style-type: none;
            padding: 0;
            margin: 20px auto;
            max-width: 500px;
            width: 100%;
            animation: fadeIn 2s ease-out;
        }
        li {
            background: rgba(0, 0, 0, 0.8);
            margin: 5px 0;
            padding: 15px;
            border-radius: 5px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.3);
            transition: transform 0.3s ease;
        }
        li:hover {
            transform: scale(1.05);
        }
        @keyframes fadeInDown {
            from {
                opacity: 0;
                transform: translateY(-50px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }
        @keyframes fadeInUp {
            from {
                opacity: 0;
                transform: translateY(50px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }
        @keyframes fadeIn {
            from {
                opacity: 0;
            }
            to {
                opacity: 1;
            }
        }
    </style>
</head>
<body>
    <video id="background-video" autoplay muted loop>
        <source src="first.mp4" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    <div class="content">
        <h1>Movie Recommendation System</h1>
        <form method="post">
            <label for="movie">Enter a movie title:</label>
            <input type="text" id="movie" name="movie" placeholder="e.g. The Matrix">
            <input type="submit" value="Get Recommendations">
        </form>
        {% if recommendations %}
            <h2>Recommendations:</h2>
            <ul>
            {% for recommendation in recommendations %}
                <li>{{ recommendation }}</li>
            {% endfor %}
            </ul>
        {% endif %}
    </div>
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def home():
    recommendations = None
    if request.method == "POST":
        movie = request.form["movie"]
        recommendations = rec_sys.get_recommendations(movie, rec_sys.cosine_sim2).tolist()
    return render_template_string(html_template, recommendations=recommendations)

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Jul/2024 15:25:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2024 15:25:39] "GET /first.mp4 HTTP/1.1" 404 -
127.0.0.1 - - [06/Jul/2024 16:13:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2024 16:13:46] "GET /first.mp4 HTTP/1.1" 404 -
127.0.0.1 - - [06/Jul/2024 16:16:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2024 16:16:45] "GET /first.mp4 HTTP/1.1" 404 -
